Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from random import choices, choice, randint, random, shuffle
from dataclasses import dataclass
from copy import copy, deepcopy
from typing import List

import lab9_lib
import matplotlib.pyplot as plt

In [ ]:
GENOME_LENGTH = 1000

@dataclass
class Individual:
    fitness: float
    genotype: list[int]

def populate(fitness, pop_size):
    population = [
        Individual(
            genotype=[choice((0,1)) for _ in range(GENOME_LENGTH)],
            fitness=None,
        )
        for _ in range(pop_size)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)

    return population

def select_parent(pop, tour_size):
    pool = [choice(pop) for _ in range(tour_size)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual, population) -> Individual:
    offspring = copy(ind)

    best = GENOME_LENGTH
    index = -1
    for i in range(GENOME_LENGTH):
        counter = 0
        for j in range(len(population)):
            if population[j].genotype[i] == 1:
                counter += 1

        if counter < best:
            best = counter
            index = i

    if index != -1:
        pos = index
    else:
        pos = randint(0, GENOME_LENGTH-1)

    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def flip_mutation(ind: Individual, population) -> Individual:
    offspring = copy(ind)
    pos = randint(0, GENOME_LENGTH-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def swap_mutation(ind: Individual, population) -> Individual:
    offspring = copy(ind)
    pos1 = randint(0, GENOME_LENGTH-1)
    pos2 = randint(0, GENOME_LENGTH-1)

    while pos2 == pos1:
        pos2 = randint(0, GENOME_LENGTH-1)

    tmp = offspring.genotype[pos1]
    offspring.genotype[pos1] = offspring.genotype[pos2]
    offspring.genotype[pos2] = tmp

    offspring.fitness = None
    return offspring

def my_xover(ind1: Individual, ind2: Individual) -> List[Individual]:
    list = [0 for _ in range(GENOME_LENGTH)]

    for i in range(GENOME_LENGTH):
        list[i] = ind1.genotype[i] or ind2.genotype[i]

    offspring = Individual(fitness=None,
                           genotype=list)
    assert len(offspring.genotype) == GENOME_LENGTH
    return [offspring]

def one_cut_xover(ind1: Individual, ind2: Individual) -> List[Individual]:
    cut_point = randint(0, GENOME_LENGTH-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == GENOME_LENGTH
    return [offspring]

def uniform_xover(ind1: Individual, ind2: Individual) -> List[Individual]:
    list1 = [0 for _ in range(GENOME_LENGTH)]
    list2 = [0 for _ in range(GENOME_LENGTH)]

    for i in range(GENOME_LENGTH):
        if random() < 0.5:
            list1[i] = ind1.genotype[i]
            list2[i] = ind2.genotype[i]
        else:
            list1[i] = ind2.genotype[i]
            list2[i] = ind1.genotype[i]

    offspring1 = Individual(fitness=None,
                           genotype=list1)
    offspring2 = Individual(fitness=None,
                           genotype=list2)
    assert len(offspring1.genotype) == GENOME_LENGTH and len(offspring2.genotype) == GENOME_LENGTH
    return [offspring1, offspring2]

In [ ]:
def hamming_distance(population):
    total_distance = 0
    num_comp = 0

    for i in range(len(population)):
        for j in range(i+1, len(population)):
            total_distance += sum([x ^ y for x,y in zip(population[i].genotype, population[j].genotype)])
            num_comp += 1

    return total_distance/num_comp

def hamming_distance2(population):
    total_distance = 0
    num_comp = 0

    for i in range(len(population)):
        for j in range(i+1, len(population)):
            total_distance += population[i].fitness - population[j].fitness
            num_comp += 1

    return total_distance/num_comp

def hamming_distance3(values):
    total_distance = 0
    num_comp = 0

    for i in range(len(values)):
        for j in range(i+1, len(values)):
            total_distance += values[i] - values[j]
            num_comp += 1

    return total_distance/num_comp

In [ ]:
def change_xover(p_xover):
    xovers = [one_cut_xover, uniform_xover]
    
    for x in xovers:
        if x != p_xover:
            return x
        
def change_mutation(p_mutation):
    mutations = [flip_mutation, swap_mutation]
    
    for m in mutations:
        if m != p_mutation:
            return m

## EA

In [ ]:
def ea(fitness, population, num_epochs, num_generations, mutation, xover, pop_size, off_size, tour_size, mut_prob, xover_prob, elitism, 
       adapt_mut, elite_xover, adapt_xm):
    best = max(population, key=lambda i:i.fitness)
    history = []
    calls_history = []

    for epoch in range(num_epochs):
        if best.fitness == 1:
            break
        
        if adapt_mut:
            #IF THE ACTUAL POPULATION HAS SMALL DIVERSITY, I INCREASE THE MUTATION PROBABILITY TO INCENTIVATE EXPLORATION
            hd = hamming_distance(population)
            #hd = hamming_distance2(population)
            #hd = hamming_distance3(history[len(history)-num_generations:])
            if hd < 400 and mut_prob <= 0.85:
                #print("INCREASE MUTATION")
                mut_prob += 0.15
            elif hd > 800 and mut_prob >= 0.15:
                #print("DECREASE MUTATION")
                mut_prob -= 0.15

        if adapt_xm and history:
            hd = hamming_distance3(history[len(history)-num_generations:])
            #hd = hamming_distance2(population)

            if hd < 0.01:
                if random() < mut_prob:
                    #print("CHANGE MUTATION")
                    mutation = change_mutation(mutation)
                
                if random() < xover_prob:
                    #print("CHANGE XOVER")
                    xover = change_xover(xover)
        
        for generation in range(num_generations):
            if best.fitness == 1:
                break

            offspring = list()
            for counter in range(off_size):
                if random() < mut_prob:
                    p = select_parent(population, tour_size)
                    o = mutation(p, population)

                    offspring.append(o)
                
                if random() < xover_prob:
                    if elite_xover:
                        off = []
                        i = 0
                        
                        shuffle(population)
                        while i+1 < len(population):
                            el_off = xover(population[i], population[i+1])
                            
                            for o in el_off:
                                o.fitness = fitness(o.genotype)
                                off.append(o)

                            i += 2

                        off.sort(key=lambda x:x.fitness, reverse=True)
                        off = off[:2]
                    else:
                        p1 = select_parent(population, tour_size)
                        p2 = select_parent(population, tour_size)
                        off = xover(p1, p2)

                    for o in off:
                        offspring.append(o)

            for i in offspring:
                if i.fitness == None:
                    i.fitness = fitness(i.genotype)

            population.extend(offspring)

            if elitism:
                population.sort(key=lambda i: i.fitness, reverse=True)
                population = population[:pop_size]

                best = population[0]
            else:
                for p in population:
                    if p.fitness > best.fitness:
                        best = p

                population = choices(population, k=pop_size)
                  
            history.append(best.fitness)
            calls_history.append(fitness.calls)

    return (best, fitness.calls, history, calls_history)

## 1

### Single Try

In [ ]:
fitness = lab9_lib.make_problem(1)
num_epochs = 10
num_generations = 50
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15

best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=swap_mutation, xover=uniform_xover, pop_size=population_size,
                  off_size=20, tour_size=2, mut_prob=0.15, xover_prob=0.5, elitism=False, adapt_mut=True, elite_xover=True, adapt_xm=True)

print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls}")
plt.plot(calls_history, history)
plt.title("Fitness Value over Fitness Calls")
plt.xlabel("Fitness Calls")
plt.ylabel("Fitness Value")
plt.grid(True)
plt.show()

### Overall

In [ ]:
fitness = lab9_lib.make_problem(1)
num_epochs = 10
num_generations = 50
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15
xover_probability = 0.5

mutations = [flip_mutation, swap_mutation]
xovers = [one_cut_xover, uniform_xover]
off_sizes = [10, 20, 30]
tour_sizes = [2, 3]
elitism = [True, False]
adapt_mut = [True, False]
elite_xover = [True, False]
adapt_xm = [True, False]

prev_calls = 0
results = []
best_fit = []
best_calls = []
best_avg = []

for o in off_sizes:
    for m in mutations:
        for x in xovers:
            for t in tour_sizes:
                for e in elitism:
                    for a in adapt_mut:
                        for ex in elite_xover:
                            for axm in adapt_xm:
                                best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=m, 
                                                                         xover=x, pop_size=population_size,
                                                                         off_size=o, tour_size=t, mut_prob=mutation_probability,
                                                                         xover_prob=xover_probability, elitism=e, adapt_mut=a,
                                                                         elite_xover=ex, adapt_xm=axm)
                            
                            print(f"(OFFSPRING: {o}, MUTATION: {m.__name__}, XOVER: {x.__name__}, TOURNAMENT: {t}, ELITISM: {e}, ADAPTIVE MUTATION: {a}, ELITE XOVER: {ex}, ADAPT XM: {axm})")
                            print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls - prev_calls}")
                            plt.plot([x-prev_calls for x in calls_history], history)
                            plt.title("Fitness Value over Fitness Calls")
                            plt.xlabel("Fitness Calls")
                            plt.ylabel("Fitness Value")
                            plt.grid(True)
                            plt.show()

                            results.append([best, calls, o, m.__name__, x.__name__, t, e, a, ex])
                            prev_calls = calls

for r in results:
    if r[0] > best_fit[0] or not best_fit:
        best_fit = r
    
    if r[1] < best_calls[1] or not best_calls:
        best_calls = r

    if r[0]/r[1] > best_avg[0]/best_avg[1] or not best_avg:
        best_avg = r

print(best_fit)
print(best_calls)
print(best_avg)

## 2

### Single Try

In [ ]:
fitness = lab9_lib.make_problem(2)
num_epochs = 10
num_generations = 100
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15

best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=flip_mutation, xover=uniform_xover, pop_size=population_size,
                  off_size=20, tour_size=2, mut_prob=0.15, elitism=True, adapt_mut=False, elite_xover=True, adapt_xm=True)

print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls}")
plt.plot(calls_history, history)
plt.title("Fitness Value over Fitness Calls")
plt.xlabel("Fitness Calls")
plt.ylabel("Fitness Value")
plt.grid(True)
plt.show()

### Overall

In [ ]:
fitness = lab9_lib.make_problem(2)
num_epochs = 10
num_generations = 50
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15
xover_probability = 0.5

mutations = [flip_mutation, swap_mutation]
xovers = [one_cut_xover, uniform_xover]
off_sizes = [10, 20, 30]
tour_sizes = [2, 3]
elitism = [True, False]
adapt_mut = [True, False]
elite_xover = [True, False]
adapt_xm = [True, False]

prev_calls = 0
results = []
best_fit = []
best_calls = []
best_avg = []

for o in off_sizes:
    for m in mutations:
        for x in xovers:
            for t in tour_sizes:
                for e in elitism:
                    for a in adapt_mut:
                        for ex in elite_xover:
                            for axm in adapt_xm:
                                best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=m, 
                                                                         xover=x, pop_size=population_size,
                                                                         off_size=o, tour_size=t, mut_prob=mutation_probability,
                                                                         xover_prob=xover_probability, elitism=e, adapt_mut=a,
                                                                         elite_xover=ex, adapt_xm=axm)
                            
                            print(f"(OFFSPRING: {o}, MUTATION: {m.__name__}, XOVER: {x.__name__}, TOURNAMENT: {t}, ELITISM: {e}, ADAPTIVE MUTATION: {a}, ELITE XOVER: {ex}, ADAPT XM: {axm})")
                            print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls - prev_calls}")
                            plt.plot([x-prev_calls for x in calls_history], history)
                            plt.title("Fitness Value over Fitness Calls")
                            plt.xlabel("Fitness Calls")
                            plt.ylabel("Fitness Value")
                            plt.grid(True)
                            plt.show()

                            results.append([best, calls, o, m.__name__, x.__name__, t, e, a, ex])
                            prev_calls = calls

for r in results:
    if r[0] > best_fit[0] or not best_fit:
        best_fit = r
    
    if r[1] < best_calls[1] or not best_calls:
        best_calls = r

    if r[0]/r[1] > best_avg[0]/best_avg[1] or not best_avg:
        best_avg = r

print(best_fit)
print(best_calls)
print(best_avg)

## 5

### Single Try

In [ ]:
fitness = lab9_lib.make_problem(5)
num_epochs = 10
num_generations = 100
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15

best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=flip_mutation, xover=uniform_xover, pop_size=population_size,
                  off_size=20, tour_size=2, mut_prob=0.15, elitism=False, adapt_mut=True, elite_xover=True, adapt_xm=True)

print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls}")
plt.plot(calls_history, history)
plt.title("Fitness Value over Fitness Calls")
plt.xlabel("Fitness Calls")
plt.ylabel("Fitness Value")
plt.grid(True)
plt.show()

### Overall

In [ ]:
fitness = lab9_lib.make_problem(5)
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15
xover_probability = 0.5

num_epochs = [10, 20, 50]
num_generations = [10, 20, 50]
mutations = [flip_mutation, swap_mutation]
xovers = [one_cut_xover, uniform_xover]
off_sizes = [10, 20, 30]
tour_sizes = [2, 3]
elitism = [True, False]
adapt_mut = [True, False]
elite_xover = [True, False]
adapt_xm = [True, False]

prev_calls = 0
results = []
best_fit = []
best_calls = []
best_avg = []

for ne in num_epochs:
    for ng in num_generations:
        #for o in off_sizes:
        o = 20
        for m in mutations:
            for x in xovers:
                #for t in tour_sizes:
                t = 2
                for e in elitism:
                    for a in adapt_mut:
                        for ex in elite_xover:
                            for axm in adapt_xm:
                                best, calls, history, calls_history = ea(fitness, population, num_epochs=ne, num_generations=ng, 
                                                                        mutation=m, xover=x, pop_size=population_size,
                                                                        off_size=o, tour_size=t, mut_prob=mutation_probability,
                                                                        xover_prob=xover_probability, elitism=e, adapt_mut=a,
                                                                        elite_xover=ex, adapt_xm=axm)
                                
                                print(f"(EPOCHS: {ne}, GENERATIONS: {ng}, OFFSPRING: {o}, MUTATION: {m.__name__}, XOVER: {x.__name__}, TOURNAMENT: {t}, ELITISM: {e}, ADAPTIVE MUTATION: {a}, ELITE XOVER: {ex}, ADAPT XM: {axm})")
                                print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls - prev_calls}")
                                plt.plot([x-prev_calls for x in calls_history], history)
                                plt.title("Fitness Value over Fitness Calls")
                                plt.xlabel("Fitness Calls")
                                plt.ylabel("Fitness Value")
                                plt.grid(True)
                                plt.show()

                                results.append([best, calls, ne, ng, o, m.__name__, x.__name__, t, e, a, ex])
                                prev_calls = calls

for r in results:
    if r[0] > best_fit[0] or not best_fit:
        best_fit = r
    
    if r[1] < best_calls[1] or not best_calls:
        best_calls = r

    if r[0]/r[1] > best_avg[0]/best_avg[1] or not best_avg:
        best_avg = r

print(best_fit)
print(best_calls)
print(best_avg)

## 10

### Single Try

In [ ]:
fitness = lab9_lib.make_problem(10)
num_epochs = 10
num_generations = 100
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15

best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=flip_mutation, xover=uniform_xover, pop_size=population_size,
                  off_size=20, tour_size=2, mut_prob=0.15, elitism=False, adapt_mut=True, elite_xover=True, adapt_xm=True)

print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls}")
plt.plot(calls_history, history)
plt.title("Fitness Value over Fitness Calls")
plt.xlabel("Fitness Calls")
plt.ylabel("Fitness Value")
plt.grid(True)
plt.show()

### Overall

In [ ]:
fitness = lab9_lib.make_problem(10)
num_epochs = 10
num_generations = 50
population_size = 20
population = populate(fitness, population_size)
mutation_probability = 0.15
xover_probability = 0.5

mutations = [flip_mutation, swap_mutation]
xovers = [one_cut_xover, uniform_xover]
off_sizes = [10, 20, 30]
tour_sizes = [2, 3]
elitism = [True, False]
adapt_mut = [True, False]
elite_xover = [True, False]
adapt_xm = [True, False]

prev_calls = 0
results = []
best_fit = []
best_calls = []
best_avg = []

for o in off_sizes:
    for m in mutations:
        for x in xovers:
            for t in tour_sizes:
                for e in elitism:
                    for a in adapt_mut:
                        for ex in elite_xover:
                            for axm in adapt_xm:
                                best, calls, history, calls_history = ea(fitness, population, num_epochs, num_generations, mutation=m, 
                                                                         xover=x, pop_size=population_size,
                                                                         off_size=o, tour_size=t, mut_prob=mutation_probability,
                                                                         xover_prob=xover_probability, elitism=e, adapt_mut=a,
                                                                         elite_xover=ex, adapt_xm=axm)
                            
                            print(f"(OFFSPRING: {o}, MUTATION: {m.__name__}, XOVER: {x.__name__}, TOURNAMENT: {t}, ELITISM: {e}, ADAPTIVE MUTATION: {a}, ELITE XOVER: {ex}, ADAPT XM: {axm})")
                            print(f"BEST FITNESS: {best.fitness:.2%}, CALLS: {calls - prev_calls}")
                            plt.plot([x-prev_calls for x in calls_history], history)
                            plt.title("Fitness Value over Fitness Calls")
                            plt.xlabel("Fitness Calls")
                            plt.ylabel("Fitness Value")
                            plt.grid(True)
                            plt.show()

                            results.append([best, calls, o, m.__name__, x.__name__, t, e, a, ex])
                            prev_calls = calls

for r in results:
    if r[0] > best_fit[0] or not best_fit:
        best_fit = r
    
    if r[1] < best_calls[1] or not best_calls:
        best_calls = r

    if r[0]/r[1] > best_avg[0]/best_avg[1] or not best_avg:
        best_avg = r

print(best_fit)
print(best_calls)
print(best_avg)